# 환경변수 로딩

In [16]:
from dotenv import load_dotenv
import os
load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# RunnableLambda 객체 사용방법

In [2]:
from langchain_core.runnables import RunnableLambda

# 함수 정의
def func(x):
    return  x*2

# 함수를 전달인자로 넣기
runnable_1 = RunnableLambda(func)

# RunnableLambda를 통한 함수 실행
print(runnable_1.invoke(10))

20


In [8]:
# RunnableLambda 객체 사용
runnable_1 = RunnableLambda(lambda x: x*2)
print(runnable_1.invoke(10))

20


In [4]:
# runnable batch
runnable_3 = RunnableLambda(func)

print(runnable_3.batch([10, 20, 30]))

[20, 40, 60]


## 순차적으로 실행하게 하기

In [ ]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

chain = r1 | r2 # r1 값이 r2의 입력으로 전달됨

chain.invoke(10)

35

In [9]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# chain = r1 | r2  # 아래와 같은 표현
chain = RunnableSequence(r1, r2)

chain.invoke(10)

35

In [10]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# 병렬처리 , r1, r2 키는 임으로 설정해 주면 됨.
# chain = RunnableParallel(r1=r1, r2=r2)
chain = RunnableParallel(first=r1, second=r2)

chain.invoke(10)

{'first': 30, 'second': 15}

In [11]:
[{"foo":2}]*3

[{'foo': 2}, {'foo': 2}, {'foo': 2}]

In [12]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


# LCEL 문법 적용
- 방법1

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    GEMINI_API_KEY=GEMINI_API_KEY, 
    temperature=0.1)

Unexpected argument 'GEMINI_API_KEY' provided to ChatGoogleGenerativeAI.
c:\Users\Admin\miniconda3\envs\lc_env\Lib\site-packages\IPython\core\interactiveshell.py:3701: UserWarning: WARNING! GEMINI_API_KEY is not default parameter.
                GEMINI_API_KEY was transferred to model_kwargs.
                Please confirm that GEMINI_API_KEY is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트와 관련된 질문만 답변해줘.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = prompt | llm
input = {
            "context": "치킨", 
            "query": "대한민국의 치킨 시장에 대해 설명해줘."
        }

In [23]:
print(chain.invoke(input))

content="대한민국의 치킨 시장은 단순히 외식 산업을 넘어 하나의 독특한 문화 현상으로 자리 잡고 있으며, 세계적으로도 유례를 찾기 어려울 정도로 거대하고 치열한 경쟁 시장입니다.\n\n다음은 대한민국의 치킨 시장에 대한 설명입니다:\n\n1.  **압도적인 규모와 경쟁:**\n    *   치킨은 한국인이 가장 즐겨 찾는 외식 메뉴 중 하나로, 전국에 수만 개의 치킨 전문점이 운영되고 있습니다. (편의점이나 마트 등에서 판매하는 냉동/간편 치킨 제품까지 포함하면 더욱 광범위합니다.)\n    *   BBQ, BHC, 교촌치킨, 굽네치킨 등 대형 프랜차이즈 브랜드들이 시장을 주도하고 있으며, 새로운 브랜드와 개인 매장이 끊임없이 생겨나고 사라지는 초경쟁 시장입니다.\n\n2.  **다양한 메뉴와 맛:**\n    *   후라이드, 양념, 간장, 마늘, 오븐구이, 바비큐, 순살, 뼈 있는 치킨 등 상상을 초월하는 다양한 조리법과 맛을 자랑합니다. 매년 새로운 소스와 조리법이 개발되어 소비자에게 새로운 경험을 제공합니다.\n    *   매운맛, 달콤한 맛, 짭짤한 맛 등 다채로운 맛 스펙트럼을 가지고 있습니다.\n\n3.  **고도로 발달한 배달 문화:**\n    *   대한민국 치킨 시장의 핵심은 '배달'입니다. 스마트폰 앱을 통해 언제 어디서든 손쉽게 주문하고 빠른 시간 내에 배달받을 수 있는 시스템이 고도로 발달해 있습니다.\n    *   '치맥'(치킨+맥주)은 한국인의 대표적인 야식 문화이자 여가 활동으로 자리 잡았습니다.\n\n4.  **프랜차이즈 중심의 시장 구조:**\n    *   대부분의 치킨 전문점은 프랜차이즈 형태로 운영됩니다. 이는 소자본 창업 아이템으로 인기가 많아 진입 장벽이 낮은 편이지만, 그만큼 치열한 경쟁과 본사와의 관계에서 발생하는 문제점들도 존재합니다.\n\n5.  **주요 트렌드:**\n    *   **건강 지향:** 튀기지 않고 오븐에 굽거나 에어프라이어로 조리하는 '구운 치킨'이나 '에어프라이어 치킨'의 인기가 

- 방법2 RunnableLambda 사용

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트와 관련된 질문만 답변해줘.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

# 기본값을 제공하는 함수
def add_default_values(input_dict):
    return {
        "context": input_dict.get("context", "삼성전자"),
        "query": input_dict.get("query", "삼성전자의 최근 실적에 대해 알려줘.")
    }


# 체인 구성
chain = RunnableLambda(add_default_values) | prompt | llm

In [32]:
# 빈 딕셔너리로도 실행 가능
result = chain.invoke({})
print(result)

content="삼성전자의 최근 실적은 2023년 어려운 시기를 거쳐 2024년 들어 강한 회복세를 보이고 있습니다. 특히, 주력 사업인 반도체 부문(DS)이 메모리 시장 회복에 힘입어 실적 개선을 주도하고 있습니다.\n\n**2024년 1분기 잠정 실적 (2024년 4월 5일 발표 기준):**\n\n*   **매출액:** 약 71.92조 원\n*   **영업이익:** 약 6.61조 원\n\n이는 시장의 예상을 뛰어넘는 '어닝 서프라이즈'에 가까운 수치로, 전년 동기 대비 매출은 약 11.37%, 영업이익은 무려 931.87% 급증했습니다.\n\n**주요 사업 부문별 동향:**\n\n1.  **반도체(DS) 부문:**\n    *   **흑자 전환:** 2022년 4분기부터 시작된 적자에서 벗어나 약 5개 분기 만에 흑자로 전환한 것이 전체 실적 개선의 가장 큰 요인입니다.\n    *   **메모리 반도체 회복:** AI 서버 수요 증가에 따른 고대역폭 메모리(HBM) 등 고부가 제품 판매 확대와 스마트폰, PC용 일반 메모리(DRAM, NAND) 가격 상승이 주효했습니다.\n    *   **재고 감소:** 고객사의 재고 조정이 마무리되고 수요가 점차 회복되면서 재고 수준도 정상화되고 있습니다.\n\n2.  **모바일경험(MX) 부문:**\n    *   **견조한 실적 유지:** 2024년 초 출시된 갤럭시 S24 시리즈의 판매 호조가 실적을 견인했습니다. 특히 '갤럭시 AI' 기능이 소비자들에게 좋은 반응을 얻으며 프리미엄 스마트폰 시장에서 경쟁력을 유지했습니다.\n    *   **폴더블폰:** 폴더블폰 등 고수익 제품군도 꾸준히 기여하고 있습니다.\n\n3.  **디스플레이(SDC) 부문:**\n    *   중소형 OLED 패널을 중심으로 안정적인 실적을 기록하며 견조한 수익성을 유지하고 있습니다.\n\n4.  **생활가전 및 TV(VD/DA) 부문:**\n    *   글로벌 경기 둔화의 영향으로 전반적인 시장 수요는 약세를 보였으나, 프리미엄 

In [33]:
# 또는 일부 값만 override
result2 = chain.invoke({"query": "RunnableLambda에 대해 1문장으로 설명해줘"})

In [34]:
print(result2.content)

질문하신 'RunnableLambda'는 일반적인 프로그래밍 개념으로, 제공된 컨텍스트인 '삼성전자'와 직접적인 관련이 없어 답변해 드릴 수 없습니다.


# 실행을 그래프로 시각화 하기

In [26]:
chain.get_graph().print_ascii()

  +--------------------------+   
  | add_default_values_input |   
  +--------------------------+   
                *                
                *                
                *                
     +--------------------+      
     | add_default_values |      
     +--------------------+      
                *                
                *                
                *                
     +--------------------+      
     | ChatPromptTemplate |      
     +--------------------+      
                *                
                *                
                *                
   +------------------------+    
   | ChatGoogleGenerativeAI |    
   +------------------------+    
                *                
                *                
                *                
+------------------------------+ 
| ChatGoogleGenerativeAIOutput | 
+------------------------------+ 
